In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.decomposition import PCA

def carregar_dados(url, nomes_colunas):
    """Carrega os dados a partir de uma URL e retorna um DataFrame."""
    data = pd.read_csv(url, names=nomes_colunas, header=0)
    return data

def pre_processamento(X, y):
    """Realiza o pré-processamento dos dados."""
    # Imputação dos valores faltantes
    imputer = SimpleImputer(strategy='most_frequent')
    y = imputer.fit_transform(y.values.reshape(-1, 1))
    y = pd.Series(y.flatten())

    # Identificação dos tipos de atributos
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Construção dos pipelines de transformação
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Aplicação das transformações nos dados
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        sparse_threshold=0
    )
    return preprocessor, y

def avaliar_modelo(modelo, X, y, pre_processor, nome_modelo):
    """Avalia um modelo usando validação cruzada e exibe métricas de desempenho."""
    pipeline = Pipeline(steps=[
        ('pre_processor', pre_processor),
        ('classifier', modelo)
    ])
    skf = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
    y_pred = cross_val_predict(pipeline, X, y, cv=skf)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='weighted')
    
    print(f"Modelo: {nome_modelo}")
    print("Acurácia:", accuracy)
    print("F1-Score:", f1)

    # Matriz de confusão
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=np.unique(y), yticklabels=np.unique(y))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {nome_modelo}')
    plt.show()

# URL dos dados e nomes das colunas
url = "https://raw.githubusercontent.com/tmfilho/akcdata/master/data/akc-data-latest.csv"
nomes_colunas = ['raça', 'descrição', 'temperamento', 'popularidade', 'altura_mínima', 'altura_máxima',
                 'peso_mínimo', 'peso_máximo', 'expectativa_vida_mínima', 'expectativa_vida_máxima', 'grupo',
                 'frequência_escovação_valor', 'frequência_escovação_categoria', 'queda_valor', 'queda_categoria',
                 'nível_energia_valor', 'nível_energia_categoria', 'facilidade_treinamento_valor', 'facilidade_treinamento_categoria',
                 'comportamento_valor', 'categoria_comportamento']

# Carregar os dados
data = carregar_dados(url, nomes_colunas)

# Separarando os atributos de entrada e alvo
X = data.drop(columns=['categoria_comportamento'])
y = data['categoria_comportamento']

# Pré-processamento dos dados
pre_processor, y = pre_processamento(X, y)

# Modelos a serem avaliados
modelos = {
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'MLP': MLPClassifier(random_state=42)
}

# Avaliar modelos
for nome, modelo in modelos.items():
    avaliar_modelo(modelo, X, y, pre_processor, nome)

# Modelos com PCA
for nome, modelo in modelos.items():
    pipeline_pca = Pipeline(steps=[
        ('preprocessor', pre_processor),
        ('pca', PCA(n_components=0.95)),  # Manter 95% da variância explicada
        ('classifier', modelo)
    ])
    avaliar_modelo(modelo, X, y, pre_processor, f'{nome} com PCA')


In [ ]:
""" 
    Todas importações necessárias para o pré-processamento dos dados,
    o teste de Algoritmos e avaliação dos resultados
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.decomposition import PCA

In [1]:
""" Agora a explicação de cada função utilizada para responder a questão """

"""
    Recebe como parâmetros a URL do conjunto de dados e os nomes das colunas.
    Lê o arquivo CSV utilizando o pandas e retorna um DataFrame com os dados.
"""

def carregar_dados(url, nomes_colunas):
    """Carrega os dados a partir de uma URL e retorna um DataFrame."""
    data = pd.read_csv(url, names=nomes_colunas, header=0)
    return data


"""
    No pré-processamento e recebido os Dataframes de entrada e alvo (X, y)
    Imputa valores ausentes em y utilizando o SimpleImputer com estratégia most_frequent.
    Identifica os tipos de atributos em X: numéricos (int64, float64) e categóricos (object).
    Cria pipelines de transformação para cada tipo de atributo:
    Numéricos:
    Imputa valores ausentes com estratégia median usando o SimpleImputer.
    Normaliza os dados com o StandardScaler.
    Categóricos:
    Imputa valores ausentes com estratégia most_frequent usando o SimpleImputer.
    Codifica os dados usando o OneHotEncoder e estratégia handle_unknown='ignore'.
    Combina os pipelines de transformação em um único ColumnTransformer.
    Retorna o ColumnTransformer e o DataFrame y pré-processado
"""

def pre_processamento(X, y):
    """Realiza o pré-processamento dos dados."""
    # Imputação dos valores faltantes
    imputer = SimpleImputer(strategy='most_frequent')
    y = imputer.fit_transform(y.values.reshape(-1, 1))
    y = pd.Series(y.flatten())

    # Identificação dos tipos de atributos
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Construção dos pipelines de transformação
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Aplicação das transformações nos dados
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        sparse_threshold=0
    )
    return preprocessor, y


"""
    Recebe como parâmetros o modelo a ser avaliado (modelo), os DataFrames de entrada (X) e alvo (y), o pré-processador (pre_processor) e o nome do modelo.
    Cria um pipeline que combina o pré-processador e o modelo.
    Realiza validação cruzada estratificada com 8 divisões (K=8) usando o StratifiedKFold.
    Prediz as classes para cada fold usando o cross_val_predict.
    Calcula a acurácia e o F1-Score utilizando as funções accuracy_score e f1_score, respectivamente.
    Exibe as métricas de desempenho para o modelo.
    Gera uma matriz de confusão usando o confusion_matrix e a visualiza com seaborn.
"""

def avaliar_modelo(modelo, X, y, pre_processor, nome_modelo):
    """Avalia um modelo usando validação cruzada e exibe métricas de desempenho."""
    pipeline = Pipeline(steps=[
        ('pre_processor', pre_processor),
        ('classifier', modelo)
    ])
    skf = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
    y_pred = cross_val_predict(pipeline, X, y, cv=skf)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='weighted')
    
    print(f"Modelo: {nome_modelo}")
    print("Acurácia:", accuracy)
    print("F1-Score:", f1)

    # Matriz de confusão
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=np.unique(y), yticklabels=np.unique(y))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {nome_modelo}')
    plt.show()





In [ ]:
"""
    E por último esse trecho do código é onde é chamada as funções
    explicadas anteriormentes, e passado os dados corretamentes
    para que obtenha os resultados.
"""

# URL dos dados e nomes das colunas
url = "https://raw.githubusercontent.com/tmfilho/akcdata/master/data/akc-data-latest.csv"
nomes_colunas = ['raça', 'descrição', 'temperamento', 'popularidade', 'altura_mínima', 'altura_máxima',
                 'peso_mínimo', 'peso_máximo', 'expectativa_vida_mínima', 'expectativa_vida_máxima', 'grupo',
                 'frequência_escovação_valor', 'frequência_escovação_categoria', 'queda_valor', 'queda_categoria',
                 'nível_energia_valor', 'nível_energia_categoria', 'facilidade_treinamento_valor', 'facilidade_treinamento_categoria',
                 'comportamento_valor', 'categoria_comportamento']


# Carregar os dados do csv
data = carregar_dados(url, nomes_colunas)

# Separarando os atributos de entrada e alvo
X = data.drop(columns=['categoria_comportamento'])
y = data['categoria_comportamento']

# Pré-processamento dos dados
pre_processor, y = pre_processamento(X, y)

# Modelos a serem avaliados
modelos = {
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'MLP': MLPClassifier(random_state=42)
}

# Avaliando modelos
for nome, modelo in modelos.items():
    avaliar_modelo(modelo, X, y, pre_processor, nome)

# Modelos com PCA
for nome, modelo in modelos.items():
    pipeline_pca = Pipeline(steps=[
        ('preprocessor', pre_processor),
        ('pca', PCA(n_components=0.95)),  # Manter 95% da variância explicada
        ('classifier', modelo)
    ])
    avaliar_modelo(modelo, X, y, pre_processor, f'{nome} com PCA')
